In [1]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

In [2]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [3]:
def translate_text_mbart(text: str, source_lang: str, target_lang: str):
    tokenizer.src_lang = source_lang
    encoded_text = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return translated_text

In [4]:
text_en = "For months, Ukraine's President Volodymyr Zelensky has been urging that the restrictions on the missiles, known as ATACMS, be lifted - allowing Kyiv to strike outside its own borders"
translated_to_pt = translate_text_mbart(text_en, "en_XX", "pt_XX")
print(f"Inglês para Português: {translated_to_pt}")

text_pt = "Os países do G20 chegaram a um consenso sobre a declaração final da cúpula do G20 no Rio de Janeiro, mas ainda há lacunas pendentes de discussão ao longo deste domingo… - Veja mais em https://noticias.uol.com.br/colunas/raquel-landim/2024/11/17/paises-chegam-a-um-acordo-sobre-declaracao-final-do-g-20-mas-ha-lacunas.htm?cmpid=copiaecola"
translated_to_en = translate_text_mbart(text_pt, "pt_XX", "en_XX")
print(f"Português para Inglês: {translated_to_en}")

Inglês para Português: Durante meses, o presidente da Ucrânia, Volodymyr Zelensky, tem pedido que as restrições sobre os foguetões, conhecidos como ATACMS, sejam levantadas - permitindo a Cieu atacar fora das suas próprias fronteiras
Português para Inglês: The G20 countries have come to a consensus on the final declaration of the G20 summit in Rio de Janeiro, but there are still gaps in discussion as of Sunday — see more at https://noticias.uol.com.br/colunas/raquel-landim/2024/11/17/paises-chegam-a-greement-on-the-final-declaration-of-the-g-20-but-on-the-colunas.htm?cmpid=copyaecola


In [5]:
from transformers import pipeline

In [6]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [8]:
def summarize_text(text, min_length=500, max_length=1000):
    summary = summarizer(text, max_length=2000, min_length=100, do_sample=False)

    summary_text = summary[0]['summary_text']

    if len(summary_text) > max_length:
        summary_text = summary_text[:max_length]
    elif len(summary_text) < min_length:
        summary_text = summary_text + " " + summary_text[:min_length - len(summary_text)]

    return summary_text

In [9]:
text = """
    O ministro da Fazenda, Fernando Haddad (PT), afirmou que o pacote de gastos do governo está pronto e só depende de ajustes com um ministério, o da Defesa. Segundo Haddad, o conjunto de medidas acertado com o presidente Lula (PT) é do tamanho que a área econômica elaborou.

Haddad falou ao canal CNBC Brasil, em entrevista veiculada na noite deste domingo (17).

"Está fechado com o presidente o conjunto de medidas e vamos anunciar brevemente. Está faltando resposta de um ministério, o Ministério da Defesa. Tivemos boas reuniões com ministro, com os comandantes das Forças, pedimos para incorporar o máximo possível para aproveitar o ensejo e corrigir distorções".

O ministro disse ser esperado que titulares de outras áreas pressionem para que os recursos das suas respectivas pastas sejam poupados. "Fui ministro da Educação e já reclamei também. Cada ministério defende sua agenda, mas o que estamos fazendo é para que todos possam crescer juntos. O pior dos mundos seria disputar fatias de um bolo que não aumenta."

A expectativa para o anúncio do governo sobre o pacote de gastos veio acompanhada de pressões na Esplanada dos Ministérios. Além da Defesa, dois ministros (Luiz Marinho, do Trabalho, e Carlos Lupi, da Previdência) chegaram a ameaçar, em entrevistas, deixar os cargos caso cortes avançassem sobre uma agenda social e de direitos.

Ao canal de notícias, Haddad defendeu a necessidade de fazer os ajustes nas contas públicas para garantir a sustentabilidade e credibilidade do arcabouço fiscal. "O objetivo é que o Brasil continue crescendo com sustentabilidade".

O pacote de medidas de contenção de gastos, elaborado por Haddad, deve ter um impacto de cerca de R$ 70 bilhões nas contas públicas nos primeiros dois anos. "[O pacote] tem o tamanho que a Fazenda apresentou para o presidente", disse o ministro na entrevista deste domingo.

A economia é estimada por integrantes do governo Lula em R$ 25 bilhões a R$ 30 bilhões em 2025. Já em 2026, o alívio é calculado em R$ 40 bilhões.
"""

In [ ]:
summary = summarize_text(text)
print(summary)

Your max_length is set to 2000, but your input_length is only 739. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=369)
